# Идея проекта, выборка и предобработка данных
<hr>

> TODO: ссылки на методологию анализа фейков по гистограммам, распознавание лиц в OpenCV, исходный датасет

# Обработка данных
<hr>

> TODO: объем и баланс, хар-ки данных на каждом этапе

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("../dfdc_dataframes/df_total_0_4_compact_old.json")

In [4]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / x)

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 279)


,index,filename,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2144,136,kwtkwmnapm.mp4,True,13.852911,14.513529,10.495222,14.615848,18.279569,25.949282,16.759270,...,8.673469,12.755102,12.882653,13.265306,13.265306,0.169368,0.018703,0.001577,8.820000,5.904298
552,52,czjxkqbhmy.mp4,True,36.682734,61.834083,50.506221,41.155015,31.745899,12.065057,19.189635,...,11.500000,6.750000,2.750000,4.250000,11.000000,0.308084,0.017627,0.003131,4.421405,3.245873
2831,212,lxdeqripfq.mp4,True,23.167231,28.325769,29.264592,30.333394,26.538469,29.251226,21.454580,...,19.214876,18.388430,9.090909,15.702479,11.983471,0.230253,0.019464,0.002157,7.125313,4.343048
1455,659,vjepxgipfi.mp4,True,11.069686,7.729025,10.090351,11.185148,12.422303,14.505334,9.660191,...,10.590278,8.506944,6.250000,6.770833,8.506944,0.095756,0.033824,0.001319,35.443636,10.443214
1348,552,ovogkdhknn.mp4,True,34.713472,46.518326,32.241986,26.563067,20.297130,17.195048,23.849662,...,22.265625,21.484375,19.531250,20.703125,24.218750,0.453686,0.016774,0.005981,3.093567,2.204167


In [5]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,
True,45.180811,542.103659,0.171121,0.284328,0.02479,0.003126,11.21667,5.665091


In [6]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,
False,49.182392,534.771606,0.200743,0.308146,0.022526,0.003057,8.383215,4.79785


In [7]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
index,541.650505,386.834343,0.166741,154.816162
pt_6_hist_noise_3,10.984466,8.157646,0.147675,2.826820
pt_8_hist_noise_3,11.521842,8.581345,0.146270,2.940497
overall_face_hist_noise_0.5_reversed,8.383215,11.216670,0.144565,2.833454
pt_7_hist_noise_3,12.003171,9.079802,0.138660,2.923369
pt_38_hist_noise_3,20.623249,15.610892,0.138332,5.012356
pt_37_hist_noise_3,22.177139,16.907346,0.134831,5.269793
pt_3_hist_noise_3,16.710629,12.823153,0.131628,3.887476
pt_5_hist_noise_3,14.397380,11.099173,0.129359,3.298207
pt_39_hist_noise_3,20.021426,15.439645,0.129206,4.581781


In [8]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
overall_face_hist_std_reversed,0.003057,0.003126,0.006184
overall_face_std_reversed,0.022526,0.024790,0.047316
overall_face_hist_noise_0.5,0.200743,0.171121,0.371864
overall_face_hist_noise_3,0.308146,0.284328,0.592474
pt_8_hist_noise_0.5,1.452484,1.419651,2.872135
pt_7_hist_noise_0.5,1.449724,1.464438,2.914162
pt_9_hist_noise_0.5,1.483363,1.458728,2.942091
pt_10_hist_noise_0.5,1.474690,1.757481,3.232172
pt_6_hist_noise_0.5,1.742977,1.759186,3.502163
pt_24_hist_noise_0.5,2.536658,2.184729,4.721387


In [9]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
index,154.816162,928.484848,0.166741
pt_6_hist_noise_3,2.826820,19.142112,0.147675
pt_8_hist_noise_3,2.940497,20.103188,0.146270
overall_face_hist_noise_0.5_reversed,2.833454,19.599885,0.144565
pt_7_hist_noise_3,2.923369,21.082973,0.138660
pt_38_hist_noise_3,5.012356,36.234141,0.138332
pt_37_hist_noise_3,5.269793,39.084485,0.134831
pt_3_hist_noise_3,3.887476,29.533782,0.131628
pt_5_hist_noise_3,3.298207,25.496552,0.129359
pt_39_hist_noise_3,4.581781,35.461071,0.129206


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [79]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(5).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(5).index) +
    list(df_smart.filter(like='std', axis=0).head(3).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

18


['pt_37_hist_noise_3',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_noise_3_reversed',
 'pt_38_hist_noise_3',
 'overall_face_hist_std',
 'pt_7_hist_noise_3',
 'overall_face_hist_noise_3',
 'pt_38_std',
 'pt_37_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_10_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'overall_face_hist_noise_0.5',
 'pt_37_std',
 'overall_face_hist_noise_0.5_reversed',
 'pt_8_hist_noise_3',
 'pt_39_std',
 'pt_6_hist_noise_3']

## Разбивка данных на train и test сплиты

In [80]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,pt_55_std,...,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2144,136,True,13.852911,14.513529,10.495222,14.615848,18.279569,25.949282,16.759270,15.997212,...,8.673469,12.755102,12.882653,13.265306,13.265306,0.169368,0.018703,0.001577,8.820000,5.904298
552,52,True,36.682734,61.834083,50.506221,41.155015,31.745899,12.065057,19.189635,31.783484,...,11.500000,6.750000,2.750000,4.250000,11.000000,0.308084,0.017627,0.003131,4.421405,3.245873
2831,212,True,23.167231,28.325769,29.264592,30.333394,26.538469,29.251226,21.454580,16.134175,...,19.214876,18.388430,9.090909,15.702479,11.983471,0.230253,0.019464,0.002157,7.125313,4.343048
1455,659,True,11.069686,7.729025,10.090351,11.185148,12.422303,14.505334,9.660191,9.793580,...,10.590278,8.506944,6.250000,6.770833,8.506944,0.095756,0.033824,0.001319,35.443636,10.443214
1348,552,True,34.713472,46.518326,32.241986,26.563067,20.297130,17.195048,23.849662,18.198725,...,22.265625,21.484375,19.531250,20.703125,24.218750,0.453686,0.016774,0.005981,3.093567,2.204167


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [82]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,pt_37_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3_reversed,pt_38_hist_noise_3,overall_face_hist_std,pt_7_hist_noise_3,overall_face_hist_noise_3,pt_38_std,pt_37_hist_noise_0.5,overall_face_std_reversed,pt_10_hist_noise_0.5,overall_face_hist_std_reversed,overall_face_hist_noise_0.5,pt_37_std,overall_face_hist_noise_0.5_reversed,pt_8_hist_noise_3,pt_39_std,pt_6_hist_noise_3
2144,5.484694,0.765306,5.904298,5.229592,634.273523,0.637755,0.169368,13.227684,0.382653,0.018703,3.571429,0.001577,0.113379,14.774320,8.820000,1.403061,9.500392,1.658163
552,37.500000,9.750000,3.245873,30.000000,319.343858,20.250000,0.308084,33.834967,10.250000,0.017627,0.250000,0.003131,0.226172,48.139677,4.421405,18.750000,31.076059,7.500000
2831,19.628099,7.024793,4.343048,21.074380,463.571613,18.595041,0.230253,30.450214,4.545455,0.019464,0.206612,0.002157,0.140345,29.577675,7.125313,9.710744,36.033055,18.801653
1455,5.902778,0.520833,10.443214,4.513889,758.192549,5.208333,0.095756,9.580049,0.868056,0.033824,0.000000,0.001319,0.028214,11.604834,35.443636,5.034722,13.373069,5.034722
1348,28.906250,5.078125,2.204167,33.984375,167.195910,32.421875,0.453686,27.676642,12.109375,0.016774,7.812500,0.005981,0.323251,24.627097,3.093567,28.125000,28.888634,25.781250


In [83]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [84]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [85]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=42)

In [86]:
pd.DataFrame(X_train).head()

,pt_37_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3_reversed,pt_38_hist_noise_3,overall_face_hist_std,pt_7_hist_noise_3,overall_face_hist_noise_3,pt_38_std,pt_37_hist_noise_0.5,overall_face_std_reversed,pt_10_hist_noise_0.5,overall_face_hist_std_reversed,overall_face_hist_noise_0.5,pt_37_std,overall_face_hist_noise_0.5_reversed,pt_8_hist_noise_3,pt_39_std,pt_6_hist_noise_3
2266,22.222222,2.430556,4.445620,15.625000,364.042161,19.444444,0.224941,22.659363,6.597222,0.014548,0.347222,0.002747,0.153369,34.412580,6.520242,12.152778,40.915531,13.541667
922,8.950617,1.543210,2.686422,5.555556,276.018992,2.160494,0.372242,8.842176,1.543210,0.022905,0.308642,0.003623,0.245885,10.835663,4.066944,11.419753,13.676152,0.925926
558,15.495868,2.685950,4.889735,11.776860,471.488120,17.975207,0.204510,15.284963,2.892562,0.018752,2.272727,0.002121,0.117945,20.870745,8.478532,11.776860,29.948456,10.743802
1620,4.338843,0.206612,8.614513,5.165289,608.040275,3.925620,0.116083,7.360402,1.446281,0.035533,6.404959,0.001645,0.035955,9.695293,27.812571,7.438017,9.270627,3.512397
1983,21.745562,5.325444,7.016788,21.745562,1114.859948,8.284024,0.142515,43.093966,8.727811,0.020853,0.000000,0.000897,0.115194,50.693916,8.681026,6.952663,42.585094,9.911243


In [87]:
y_train.head()

2266    False
922      True
558      True
1620     True
1983    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [88]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [89]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [90]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [91]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[55 50]
 [20 73]]
               precision    recall  f1-score   support

 class "real"       0.73      0.52      0.61       105
class "fakes"       0.59      0.78      0.68        93

     accuracy                           0.65       198
    macro avg       0.66      0.65      0.64       198
 weighted avg       0.67      0.65      0.64       198



### SVC (f1=0.54) [to be modified]

In [92]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[43 62]
 [20 73]]
               precision    recall  f1-score   support

 class "real"       0.68      0.41      0.51       105
class "fakes"       0.54      0.78      0.64        93

     accuracy                           0.59       198
    macro avg       0.61      0.60      0.58       198
 weighted avg       0.62      0.59      0.57       198



### LogisticRegression (f1=0.64) [to be modified]

In [93]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[60 45]
 [26 67]]
               precision    recall  f1-score   support

 class "real"       0.70      0.57      0.63       105
class "fakes"       0.60      0.72      0.65        93

     accuracy                           0.64       198
    macro avg       0.65      0.65      0.64       198
 weighted avg       0.65      0.64      0.64       198



### NN - MLPClassifier (f1=0.56) [to be modified]

In [94]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(30,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[57 48]
 [32 61]]
               precision    recall  f1-score   support

 class "real"       0.64      0.54      0.59       105
class "fakes"       0.56      0.66      0.60        93

     accuracy                           0.60       198
    macro avg       0.60      0.60      0.60       198
 weighted avg       0.60      0.60      0.60       198



### RandomForestClassifier (good, f1=0.60) [to be modified] 

In [95]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[49 56]
 [31 62]]
               precision    recall  f1-score   support

 class "real"       0.61      0.47      0.53       105
class "fakes"       0.53      0.67      0.59        93

     accuracy                           0.56       198
    macro avg       0.57      0.57      0.56       198
 weighted avg       0.57      0.56      0.56       198



### DecisionTreeClassifier (f1=0.56) [to be modified]

In [96]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=3, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[38 67]
 [22 71]]
               precision    recall  f1-score   support

 class "real"       0.63      0.36      0.46       105
class "fakes"       0.51      0.76      0.61        93

     accuracy                           0.55       198
    macro avg       0.57      0.56      0.54       198
 weighted avg       0.58      0.55      0.53       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [97]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=6.5, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[  0 105]
 [  0  93]]
               precision    recall  f1-score   support

 class "real"        nan      0.00       nan       105
class "fakes"       0.47      1.00      0.64        93

     accuracy                           0.47       198
    macro avg       0.47      0.50      0.64       198
 weighted avg       0.47      0.47      0.64       198



### KNeighborsClassifier (f1=0.60) [to be modified]

In [98]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=7, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)

[[56 49]
 [31 62]]
               precision    recall  f1-score   support

 class "real"       0.64      0.53      0.58       105
class "fakes"       0.56      0.67      0.61        93

     accuracy                           0.60       198
    macro avg       0.60      0.60      0.60       198
 weighted avg       0.60      0.60      0.59       198

